# Phase 2: Topological Verification - b3 = 77

**Objective**: Verify that our G2-structure lives on a manifold with b3 = 77.

## What We Have (Phase 1)
- G2-structure with ||T|| = 0.00140 < epsilon_0 = 0.0288 ✓
- det(g) = 65/32 exact ✓
- Positive definite metric ✓

## What We Need (Phase 2)
- **b3 = 77**: Third Betti number = dim(H^3) = 77 harmonic 3-forms
- **Spectral gap**: Clear separation after eigenvalue #77
- **3 generations**: Clustering structure for N_gen = 3

## Methods
1. **High-resolution Hodge Laplacian**: Discrete approximation on fine mesh
2. **Persistent Homology**: Topological data analysis with Gudhi
3. **Spectral clustering**: Verify 3-generation structure

In [ ]:
# Install dependencies
!pip install -q gudhi torch numpy scipy matplotlib scikit-learn

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
from scipy.sparse.linalg import eigsh
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
import math
import json
from typing import Dict, List, Tuple
from tqdm.auto import tqdm

try:
    import gudhi as gd
    GUDHI_AVAILABLE = True
    print("Gudhi available for persistent homology")
except ImportError:
    GUDHI_AVAILABLE = False
    print("Gudhi not available - will use spectral methods only")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

# GIFT targets
TARGET_B3 = 77
TARGET_B2 = 21
TARGET_DET = 65.0 / 32.0

## 1. Load Trained Model

In [ ]:
# Model definition
class FourierFeatures(nn.Module):
    def __init__(self, input_dim=7, num_frequencies=64, scale=0.5):
        super().__init__()
        self.output_dim = 2 * num_frequencies
        B = torch.randn(num_frequencies, input_dim) * scale
        self.register_buffer('B', B)
    
    def forward(self, x):
        x_proj = 2 * math.pi * torch.matmul(x, self.B.T)
        return torch.cat([torch.sin(x_proj), torch.cos(x_proj)], dim=-1)


def standard_g2_phi(device=None):
    phi = torch.zeros(35, device=device, dtype=torch.float64)
    G2_INDICES = [(0,1,2), (0,3,4), (0,5,6), (1,3,5), (1,4,6), (2,3,6), (2,4,5)]
    G2_SIGNS = [1, 1, 1, 1, -1, -1, -1]
    
    def to_index(i, j, k):
        count = 0
        for a in range(7):
            for b in range(a + 1, 7):
                for c in range(b + 1, 7):
                    if a == i and b == j and c == k:
                        return count
                    count += 1
        return -1
    
    for indices, sign in zip(G2_INDICES, G2_SIGNS):
        idx = to_index(*indices)
        if idx >= 0:
            phi[idx] = float(sign)
    return phi


class G2LowTorsionNet(nn.Module):
    def __init__(self, hidden_dims=[256, 512, 512, 512, 256], num_frequencies=64, 
                 fourier_scale=0.5, perturbation_scale=0.05, device=None):
        super().__init__()
        self.device = device or torch.device('cpu')
        self.fourier = FourierFeatures(7, num_frequencies, fourier_scale)
        
        layers = []
        prev_dim = self.fourier.output_dim
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.LayerNorm(hidden_dim),
                nn.SiLU(),
            ])
            prev_dim = hidden_dim
        self.mlp = nn.Sequential(*layers)
        self.output_layer = nn.Linear(prev_dim, 35)
        self.bias = nn.Parameter(standard_g2_phi(self.device))
        self.scale = nn.Parameter(torch.ones(35, device=self.device, dtype=torch.float64) * perturbation_scale)
    
    def forward(self, x):
        x_enc = self.fourier(x)
        h = self.mlp(x_enc)
        phi_raw = self.output_layer(h)
        return phi_raw * self.scale + self.bias


# Load model
model = G2LowTorsionNet(device=device).to(device).double()

try:
    checkpoint = torch.load('g2_low_torsion_model.pt', map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Model loaded!")
    print(f"  det(g) = {checkpoint['final_det_g']:.6f}")
    print(f"  ||T|| = {checkpoint['torsion_scaled']:.6f}")
except Exception as e:
    print(f"Upload g2_low_torsion_model.pt: {e}")

model.eval()

## 2. Build High-Resolution Point Cloud

In [ ]:
def sample_manifold_points(model, n_points: int = 8192, method: str = 'uniform') -> torch.Tensor:
    """
    Sample points on the G2 manifold.
    
    For high-quality spectrum, we need well-distributed points.
    """
    if method == 'uniform':
        # Uniform in [-1, 1]^7
        points = torch.rand(n_points, 7, device=device, dtype=torch.float64) * 2 - 1
    
    elif method == 'sobol':
        # Sobol sequence for better space-filling
        from torch.quasirandom import SobolEngine
        sobol = SobolEngine(dimension=7, scramble=True)
        points = sobol.draw(n_points).to(device).double() * 2 - 1
    
    elif method == 'latin_hypercube':
        # Latin hypercube sampling
        from scipy.stats import qmc
        sampler = qmc.LatinHypercube(d=7)
        sample = sampler.random(n=n_points)
        points = torch.from_numpy(sample * 2 - 1).to(device).double()
    
    return points


# Sample points
N_POINTS = 8192  # High resolution
print(f"Sampling {N_POINTS} points...")

try:
    points = sample_manifold_points(model, N_POINTS, method='sobol')
    print("Using Sobol sequence")
except:
    points = sample_manifold_points(model, N_POINTS, method='uniform')
    print("Using uniform sampling")

print(f"Points shape: {points.shape}")

In [ ]:
def expand_phi_to_tensor(phi_components):
    """Expand 35 components to full (N, 7, 7, 7) tensor."""
    N = phi_components.shape[0]
    phi_full = torch.zeros(N, 7, 7, 7, device=phi_components.device, dtype=phi_components.dtype)
    
    idx = 0
    for i in range(7):
        for j in range(i + 1, 7):
            for k in range(j + 1, 7):
                val = phi_components[:, idx]
                phi_full[:, i, j, k] = val
                phi_full[:, i, k, j] = -val
                phi_full[:, j, i, k] = -val
                phi_full[:, j, k, i] = val
                phi_full[:, k, i, j] = val
                phi_full[:, k, j, i] = -val
                idx += 1
    return phi_full


def compute_metric(phi_full):
    """Compute g_ij from phi."""
    return torch.einsum('...ikl,...jkl->...ij', phi_full, phi_full) / 6.0


# Compute phi and metric at all points
print("Computing G2 structure at all points...")

with torch.no_grad():
    phi = model(points)  # (N, 35)
    phi_full = expand_phi_to_tensor(phi)  # (N, 7, 7, 7)
    metric = compute_metric(phi_full)  # (N, 7, 7)

print(f"phi shape: {phi.shape}")
print(f"metric shape: {metric.shape}")
print(f"det(g) mean: {torch.det(metric).mean().item():.6f}")

## 3. Build Graph Laplacian for 3-Forms

The Hodge Laplacian on 3-forms: $\Delta_3 = d d^* + d^* d$

We approximate this using a discrete graph Laplacian weighted by the metric.

In [ ]:
def build_knn_graph(points: torch.Tensor, k: int = 20) -> Tuple[np.ndarray, np.ndarray]:
    """
    Build k-nearest neighbor graph.
    
    Returns:
        indices: (N, k) neighbor indices
        distances: (N, k) distances to neighbors
    """
    points_np = points.cpu().numpy()
    
    nbrs = NearestNeighbors(n_neighbors=k+1, algorithm='auto').fit(points_np)
    distances, indices = nbrs.kneighbors(points_np)
    
    # Remove self-connections
    return indices[:, 1:], distances[:, 1:]


# Build k-NN graph
K_NEIGHBORS = 30  # More neighbors for better approximation
print(f"Building {K_NEIGHBORS}-NN graph...")

neighbor_idx, neighbor_dist = build_knn_graph(points, k=K_NEIGHBORS)
print(f"Graph built: {neighbor_idx.shape}")
print(f"Mean distance: {neighbor_dist.mean():.4f}")

In [ ]:
def build_hodge_laplacian_3forms(points: torch.Tensor, phi: torch.Tensor, 
                                  metric: torch.Tensor, neighbor_idx: np.ndarray,
                                  neighbor_dist: np.ndarray) -> sparse.csr_matrix:
    """
    Build discrete Hodge Laplacian for 3-forms.
    
    For 3-forms on a 7-manifold:
    - dim(Lambda^3) = C(7,3) = 35
    - Total DOFs = N_points * 35
    
    The Laplacian couples neighboring points through the metric.
    """
    N = points.shape[0]
    n_3forms = 35
    total_dof = N * n_3forms
    
    print(f"Building Hodge Laplacian: {total_dof} DOFs")
    
    # Convert to numpy
    phi_np = phi.cpu().numpy()  # (N, 35)
    metric_np = metric.cpu().numpy()  # (N, 7, 7)
    
    # Build sparse Laplacian
    # Using weighted graph Laplacian: L = D - W
    # Weight w_ij = exp(-d_ij^2 / sigma^2) * metric_coupling
    
    sigma = neighbor_dist.mean() * 1.5
    
    rows = []
    cols = []
    data = []
    
    print("Computing Laplacian entries...")
    for i in tqdm(range(N)):
        # Diagonal block: sum of weights
        diag_weight = 0.0
        
        for k_idx, j in enumerate(neighbor_idx[i]):
            d_ij = neighbor_dist[i, k_idx]
            w_ij = np.exp(-d_ij**2 / sigma**2)
            
            # Metric coupling between phi at i and j
            # Use inner product of 3-forms: <phi_i, phi_j>_g
            phi_i = phi_np[i]  # (35,)
            phi_j = phi_np[j]  # (35,)
            
            # Simple coupling: correlation of phi values
            coupling = np.abs(np.dot(phi_i, phi_j)) / (np.linalg.norm(phi_i) * np.linalg.norm(phi_j) + 1e-10)
            
            weight = w_ij * (0.5 + 0.5 * coupling)
            diag_weight += weight
            
            # Off-diagonal: -weight for each 3-form component
            for f in range(n_3forms):
                row = i * n_3forms + f
                col = j * n_3forms + f
                rows.append(row)
                cols.append(col)
                data.append(-weight)
        
        # Diagonal entries
        for f in range(n_3forms):
            row = i * n_3forms + f
            rows.append(row)
            cols.append(row)
            data.append(diag_weight)
    
    # Create sparse matrix
    laplacian = sparse.csr_matrix((data, (rows, cols)), shape=(total_dof, total_dof))
    
    # Symmetrize
    laplacian = (laplacian + laplacian.T) / 2
    
    print(f"Laplacian shape: {laplacian.shape}")
    print(f"Non-zeros: {laplacian.nnz}")
    
    return laplacian


# Build Laplacian
laplacian = build_hodge_laplacian_3forms(points, phi, metric, neighbor_idx, neighbor_dist)

## 4. Compute Spectrum and Find Gap at b3 = 77

In [ ]:
def compute_spectrum(laplacian: sparse.csr_matrix, n_eigenvalues: int = 150) -> np.ndarray:
    """
    Compute smallest eigenvalues of Laplacian.
    
    For b3 = 77, we expect 77 eigenvalues near 0 (harmonic forms).
    """
    print(f"Computing {n_eigenvalues} smallest eigenvalues...")
    
    # Shift-invert mode for small eigenvalues
    eigenvalues, eigenvectors = eigsh(
        laplacian, 
        k=n_eigenvalues, 
        which='SM',  # Smallest magnitude
        tol=1e-8,
        maxiter=5000,
    )
    
    # Sort by magnitude
    idx = np.argsort(np.abs(eigenvalues))
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    
    print(f"Computed {len(eigenvalues)} eigenvalues")
    print(f"Range: [{eigenvalues.min():.6e}, {eigenvalues.max():.6e}]")
    
    return eigenvalues, eigenvectors


# Compute spectrum
N_EIGENVALUES = 150  # More than 77 to see the gap
eigenvalues, eigenvectors = compute_spectrum(laplacian, n_eigenvalues=N_EIGENVALUES)

In [ ]:
def analyze_spectral_gap(eigenvalues: np.ndarray, target_b3: int = 77) -> Dict:
    """
    Analyze spectrum to find gap at b3.
    
    For a manifold with b3 = 77:
    - First 77 eigenvalues should be small (near-harmonic forms)
    - Clear gap after eigenvalue #77
    """
    # Take absolute values (eigenvalues should be non-negative)
    eigs = np.abs(eigenvalues)
    eigs = np.sort(eigs)
    
    # Compute gaps
    gaps = np.diff(eigs)
    
    # Find largest gaps
    gap_order = np.argsort(gaps)[::-1]
    
    # Statistics
    mean_gap = gaps.mean()
    std_gap = gaps.std()
    
    results = {
        'n_eigenvalues': len(eigs),
        'eigenvalues': eigs.tolist(),
        'gaps': gaps.tolist(),
        'mean_gap': float(mean_gap),
        'std_gap': float(std_gap),
        'largest_gaps': [],
    }
    
    print("\nSpectral Gap Analysis")
    print("="*60)
    print(f"Target b3: {target_b3}")
    print(f"Mean gap: {mean_gap:.6e}")
    print(f"Std gap: {std_gap:.6e}")
    print("\nLargest gaps:")
    
    for i, idx in enumerate(gap_order[:10]):
        position = idx + 1  # Gap after eigenvalue #(idx+1)
        gap_value = gaps[idx]
        gap_ratio = gap_value / mean_gap
        
        marker = "" 
        if position == target_b3:
            marker = " <-- TARGET b3=77!"
        elif abs(position - target_b3) <= 3:
            marker = f" (near target, off by {position - target_b3})"
        
        print(f"  #{i+1}: Gap after eigenvalue {position}: {gap_value:.6e} ({gap_ratio:.1f}x mean){marker}")
        
        results['largest_gaps'].append({
            'position': int(position),
            'gap_value': float(gap_value),
            'gap_ratio': float(gap_ratio),
        })
    
    # Check gap at target position
    if target_b3 < len(gaps):
        target_gap = gaps[target_b3 - 1]
        target_ratio = target_gap / mean_gap
        results['gap_at_target'] = {
            'position': target_b3,
            'gap_value': float(target_gap),
            'gap_ratio': float(target_ratio),
        }
        print(f"\nGap at target b3={target_b3}: {target_gap:.6e} ({target_ratio:.1f}x mean)")
    
    # Determine best candidate for b3
    best_gap_pos = gap_order[0] + 1
    results['best_candidate_b3'] = int(best_gap_pos)
    results['distance_from_target'] = int(abs(best_gap_pos - target_b3))
    
    # Verdict
    if best_gap_pos == target_b3:
        results['verdict'] = 'EXACT_MATCH'
        print(f"\n*** EXACT MATCH: Largest gap at position {target_b3} = b3 ***")
    elif abs(best_gap_pos - target_b3) <= 5:
        results['verdict'] = 'CLOSE_MATCH'
        print(f"\nClose match: Largest gap at {best_gap_pos}, target is {target_b3}")
    else:
        results['verdict'] = 'MISMATCH'
        print(f"\nMismatch: Largest gap at {best_gap_pos}, target is {target_b3}")
    
    return results


# Analyze spectrum
spectral_results = analyze_spectral_gap(eigenvalues, target_b3=TARGET_B3)

In [ ]:
# Plot spectrum
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

eigs = np.abs(np.sort(eigenvalues))
gaps = np.diff(eigs)

# 1. Eigenvalue spectrum
ax = axes[0, 0]
ax.semilogy(range(1, len(eigs)+1), eigs, 'b.-', markersize=3)
ax.axvline(x=TARGET_B3, color='r', linestyle='--', linewidth=2, label=f'Target b3={TARGET_B3}')
ax.axvline(x=spectral_results['best_candidate_b3'], color='g', linestyle=':', linewidth=2, 
           label=f'Best gap at {spectral_results["best_candidate_b3"]}')
ax.set_xlabel('Eigenvalue index')
ax.set_ylabel('Eigenvalue (log scale)')
ax.set_title('Hodge Laplacian Spectrum')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Gaps
ax = axes[0, 1]
ax.bar(range(1, len(gaps)+1), gaps, width=1, alpha=0.7)
ax.axvline(x=TARGET_B3, color='r', linestyle='--', linewidth=2, label=f'Target b3={TARGET_B3}')
ax.set_xlabel('Gap position')
ax.set_ylabel('Gap size')
ax.set_title('Spectral Gaps')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Zoom around b3
ax = axes[1, 0]
zoom_start = max(0, TARGET_B3 - 30)
zoom_end = min(len(eigs), TARGET_B3 + 30)
ax.semilogy(range(zoom_start+1, zoom_end+1), eigs[zoom_start:zoom_end], 'b.-', markersize=5)
ax.axvline(x=TARGET_B3, color='r', linestyle='--', linewidth=2, label=f'Target b3={TARGET_B3}')
ax.set_xlabel('Eigenvalue index')
ax.set_ylabel('Eigenvalue (log scale)')
ax.set_title(f'Zoom: Eigenvalues {zoom_start+1}-{zoom_end}')
ax.legend()
ax.grid(True, alpha=0.3)

# 4. Gap ratios
ax = axes[1, 1]
gap_ratios = gaps / gaps.mean()
ax.bar(range(1, len(gap_ratios)+1), gap_ratios, width=1, alpha=0.7)
ax.axvline(x=TARGET_B3, color='r', linestyle='--', linewidth=2, label=f'Target b3={TARGET_B3}')
ax.axhline(y=3, color='orange', linestyle=':', label='3x threshold')
ax.set_xlabel('Gap position')
ax.set_ylabel('Gap ratio (vs mean)')
ax.set_title('Normalized Gap Ratios')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('spectral_analysis_b3.png', dpi=150)
plt.show()

print(f"\nPlot saved to spectral_analysis_b3.png")

## 5. Persistent Homology with Gudhi

In [ ]:
def compute_persistent_homology(points: np.ndarray, max_dimension: int = 3, 
                                 max_edge_length: float = 1.0) -> Dict:
    """
    Compute persistent homology using Gudhi.
    
    Returns Betti numbers at various filtration scales.
    """
    if not GUDHI_AVAILABLE:
        return {'error': 'Gudhi not available'}
    
    print(f"Computing persistent homology...")
    print(f"  Points: {len(points)}")
    print(f"  Max dimension: {max_dimension}")
    print(f"  Max edge length: {max_edge_length}")
    
    # Build Rips complex
    rips = gd.RipsComplex(points=points, max_edge_length=max_edge_length)
    simplex_tree = rips.create_simplex_tree(max_dimension=max_dimension + 1)
    
    print(f"  Simplices: {simplex_tree.num_simplices()}")
    
    # Compute persistence
    simplex_tree.compute_persistence()
    
    # Get Betti numbers
    betti = simplex_tree.betti_numbers()
    
    # Get persistence pairs
    persistence = simplex_tree.persistence()
    
    results = {
        'betti_numbers': betti,
        'num_simplices': simplex_tree.num_simplices(),
        'persistence_pairs': [(dim, (birth, death)) for dim, (birth, death) in persistence],
    }
    
    print(f"\nBetti numbers: {betti}")
    print(f"  b0 = {betti[0]} (connected components)")
    print(f"  b1 = {betti[1] if len(betti) > 1 else 0} (1-cycles)")
    print(f"  b2 = {betti[2] if len(betti) > 2 else 0} (2-cycles) [target: {TARGET_B2}]")
    print(f"  b3 = {betti[3] if len(betti) > 3 else 0} (3-cycles) [target: {TARGET_B3}]")
    
    return results


if GUDHI_AVAILABLE:
    # Use subset of points (Gudhi can be slow for large point clouds)
    n_subset = min(2000, N_POINTS)
    subset_idx = np.random.choice(N_POINTS, n_subset, replace=False)
    points_subset = points[subset_idx].cpu().numpy()
    
    homology_results = compute_persistent_homology(
        points_subset, 
        max_dimension=3,
        max_edge_length=0.8
    )
else:
    print("Skipping persistent homology (Gudhi not installed)")
    homology_results = {'error': 'Gudhi not available'}

In [ ]:
# Plot persistence diagram
if GUDHI_AVAILABLE and 'persistence_pairs' in homology_results:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Persistence diagram
    ax = axes[0]
    colors = ['blue', 'orange', 'green', 'red']
    for dim, (birth, death) in homology_results['persistence_pairs']:
        if death == float('inf'):
            death = 2.0  # Cap for visualization
        ax.scatter(birth, death, c=colors[dim % len(colors)], s=20, alpha=0.6, label=f'H{dim}' if birth == 0 else '')
    
    ax.plot([0, 2], [0, 2], 'k--', alpha=0.3)
    ax.set_xlabel('Birth')
    ax.set_ylabel('Death')
    ax.set_title('Persistence Diagram')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Betti number bar chart
    ax = axes[1]
    betti = homology_results['betti_numbers']
    x = range(len(betti))
    ax.bar(x, betti, color=['blue', 'orange', 'green', 'red'][:len(betti)])
    
    # Add target lines
    if len(betti) > 2:
        ax.axhline(y=TARGET_B2, color='green', linestyle='--', alpha=0.5, label=f'Target b2={TARGET_B2}')
    if len(betti) > 3:
        ax.axhline(y=TARGET_B3, color='red', linestyle='--', alpha=0.5, label=f'Target b3={TARGET_B3}')
    
    ax.set_xlabel('Dimension')
    ax.set_ylabel('Betti number')
    ax.set_title('Betti Numbers')
    ax.set_xticks(x)
    ax.set_xticklabels([f'b{i}' for i in x])
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('persistent_homology.png', dpi=150)
    plt.show()

## 6. Three-Generation Structure

In [ ]:
def analyze_generation_structure(eigenvalues: np.ndarray, eigenvectors: np.ndarray,
                                  b3: int = 77, n_gen: int = 3) -> Dict:
    """
    Analyze if eigenvalues cluster into 3 generations.
    
    For GIFT, the 77 harmonic 3-forms should cluster into 3 families
    corresponding to 3 fermion generations.
    """
    # Take first b3 eigenvalues (harmonic forms)
    eigs = np.abs(eigenvalues[:b3])
    
    # K-means clustering into n_gen groups
    kmeans = KMeans(n_clusters=n_gen, random_state=42, n_init=10)
    labels = kmeans.fit_predict(eigs.reshape(-1, 1))
    
    # Analyze clusters
    cluster_sizes = [np.sum(labels == i) for i in range(n_gen)]
    cluster_centers = kmeans.cluster_centers_.flatten()
    
    # Sort by cluster center
    order = np.argsort(cluster_centers)
    cluster_sizes = [cluster_sizes[i] for i in order]
    cluster_centers = cluster_centers[order]
    
    results = {
        'n_generations': n_gen,
        'cluster_sizes': cluster_sizes,
        'cluster_centers': cluster_centers.tolist(),
        'total_forms': sum(cluster_sizes),
    }
    
    print("\nThree-Generation Analysis")
    print("="*60)
    print(f"Clustering first {b3} eigenvalues into {n_gen} groups:\n")
    
    for i, (size, center) in enumerate(zip(cluster_sizes, cluster_centers)):
        print(f"  Generation {i+1}: {size} forms (center: {center:.6e})")
    
    # Check if roughly equal
    expected_size = b3 // n_gen
    size_variance = np.var(cluster_sizes)
    balanced = all(abs(s - expected_size) <= expected_size * 0.3 for s in cluster_sizes)
    
    results['expected_size'] = expected_size
    results['size_variance'] = float(size_variance)
    results['balanced'] = balanced
    
    print(f"\nExpected ~{expected_size} forms per generation")
    print(f"Balance: {'GOOD' if balanced else 'UNBALANCED'}")
    
    return results


# Analyze generation structure
n_harmonic = min(TARGET_B3, len(eigenvalues))
generation_results = analyze_generation_structure(eigenvalues, eigenvectors, b3=n_harmonic, n_gen=3)

## 7. Final Topology Certificate

In [ ]:
# Compile topology certificate
topology_certificate = {
    'type': 'G2_TOPOLOGY_CERTIFICATE',
    'version': '2.0',
    'targets': {
        'b2': TARGET_B2,
        'b3': TARGET_B3,
        'n_gen': 3,
    },
    'mesh': {
        'n_points': N_POINTS,
        'k_neighbors': K_NEIGHBORS,
    },
    'spectral_analysis': {
        'n_eigenvalues': len(eigenvalues),
        'best_gap_position': spectral_results['best_candidate_b3'],
        'target_gap_position': TARGET_B3,
        'distance_from_target': spectral_results['distance_from_target'],
        'verdict': spectral_results['verdict'],
        'gap_at_target': spectral_results.get('gap_at_target', {}),
        'largest_gaps': spectral_results['largest_gaps'][:5],
    },
    'persistent_homology': homology_results if GUDHI_AVAILABLE else {'status': 'not_computed'},
    'generation_structure': generation_results,
}

# Determine overall status
if spectral_results['verdict'] == 'EXACT_MATCH':
    topology_certificate['status'] = 'VERIFIED'
    topology_certificate['conclusion'] = f"b3 = {TARGET_B3} VERIFIED: Spectral gap at exact position."
elif spectral_results['verdict'] == 'CLOSE_MATCH':
    topology_certificate['status'] = 'LIKELY_VERIFIED'
    topology_certificate['conclusion'] = (
        f"b3 = {TARGET_B3} LIKELY: Gap at position {spectral_results['best_candidate_b3']}, "
        f"within {spectral_results['distance_from_target']} of target."
    )
else:
    topology_certificate['status'] = 'INCONCLUSIVE'
    topology_certificate['conclusion'] = (
        f"b3 = {TARGET_B3} INCONCLUSIVE: Best gap at {spectral_results['best_candidate_b3']}. "
        f"May need higher resolution mesh."
    )

# Save certificate
with open('topology_certificate.json', 'w') as f:
    # Convert numpy arrays to lists for JSON
    def convert(obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, dict):
            return {k: convert(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [convert(v) for v in obj]
        elif isinstance(obj, (np.integer, np.floating)):
            return float(obj)
        return obj
    
    json.dump(convert(topology_certificate), f, indent=2)

print("\n" + "="*60)
print("TOPOLOGY CERTIFICATE")
print("="*60)
print(f"\nStatus: {topology_certificate['status']}")
print(f"\n{topology_certificate['conclusion']}")
print(f"\nSaved to topology_certificate.json")

In [ ]:
# Print full certificate
print(json.dumps(topology_certificate, indent=2, default=str))